In [1]:
!pip install geopy

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import  Nominatim

C:\Users\Aman saini\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
df=pd.read_csv('F:/r file/project/GlobalTerrosim/globalterrorism.csv',encoding='ISO-8859-1',sep=',')

C:\Users\Aman saini\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4,6,26,27,46,47,48,59,61,69,70,87,88,94) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
NameConv={
        'iyear':'Year',
        'imonth':'Month',
        'iday': 'Day',
        'country_txt':'Country',
        'region_txt':'Region',
        'alternative_txt':'Alternative',
        'attacktype1_txt':'AttackType1',
        'attacktype2_txt':'AttackType2',
        'attacktype3_txt':'AttackType3',
        'targtype1_txt':'TargetType1',
        'targsubtype1_txt':'TargetSubType1',
        'natlty1_txt':'Natlty1',
        'targtype2_txt':'TargetType2',
        'targsubtype2_txt':'TargetSubType2',
        'natlty2_txt':'Natlty2',
        'targtype3_txt':'TargetType3',
        'natlty3_txt':'Natlty3',
        'claimmode_txt':'ClaimMode1',
        'claimmode2_txt':'ClaimMode2',
        'claimmode3_txt':'ClaimMode3',
        'weaptype1_txt':'WeapType1',
        'weapsubtype1_txt':'WeapSubType1',
        'weaptype2_txt':'WeapType2',
        'weapsubtype2_txt':'WeapSubType2',
        'weaptype3_txt':'WeapType3',
        'weapsubtype3_txt':'WeapSubType3',
        'weaptype4_txt':'WeapType4',
        'weapsubtype4_txt':'WeapSubType4',
        'propextent_txt':'PropeRXtent',   
        'targsubtype3_txt':'TargetSubType3'       
}

df.rename(columns=NameConv,inplace=True)

In [5]:
# Lets check the shape of dataset
## Pre shape before droppping duplicates
df.shape

(181691, 107)

In [6]:
## Lets see is there any duplicate present in data or not

df.duplicated().sum()

4550

In [7]:
df.drop_duplicates(keep='first',inplace=True)

In [8]:
# Post shape after dropping duplicates
df.shape

(177141, 107)

In [9]:
## Seems like 'gsubname','gsubname2' and 'gsubname3' are all related to Sub group name of attacker which needs to be combine into one columns.
## Same for 'gname', 'gname2' and 'gname3' are all related to Group Name of attacker which needs to be combine into one columns.
## Same for 'WeaponType1' , 'WeaponType2','WeaponType3' and 'WeaponType4' all are related to WeaponType which needs to combine into one columns.
## Same for 'WeaponSubType1' , 'WeaponSubType2','WeaponSubType3' and 'WeaponSubType4' all are related to WeaponType which needs to combine into one columns.
## Same for 'AttackType1','AttackType2' and 'AttackType3' into one column called 'AttackType'.
## Same for 'Target1','Target2' and 'Target3' into one column called 'TargetType'.
## Same for 'TargetSubType1','TargetSubType2' and 'TargetSubType3' into one column called 'TargetSubType'.
## Same for 'TargetType1','TargetType2' and ''TargetType3' into one columns called 'TargetType'
## Same for 'natlty1','natlty2' and 'natlty3' belongs to nationality of target that was attacked needs to combine into one columns called 'Nationality'.
## 'nkill''n_wounded' applying total number of casuality occur into one columns as 'Ncasuality'

In [10]:
df['GroupSubName']=df['gsubname'].astype(str)+" "+df['gsubname2'].astype(str)+" "+df['gsubname3'].astype(str)
df['GroupName']=df['gname'].astype(str)+" "+df['gname2'].astype(str)+" "+df['gname3'].astype(str)
df['WeponType']=df['WeapType1'].astype(str)+" "+df['WeapType2'].astype(str)+" "+df['WeapType3'].astype(str)+" "+df['WeapType4'].astype(str)
df['WeponSubType']=df['WeapSubType1'].astype(str)+" "+df['WeapSubType2'].astype(str)+" "+df['WeapSubType3'].astype(str)+" "+df['WeapSubType4'].astype(str)
df['AttackType']=df['AttackType1'].astype(str)+" "+df['AttackType2'].astype(str)+" "+df['AttackType3'].astype(str)
df['Target']=df['target1'].astype(str)+" "+df['target2'].astype(str)+" "+df['target3'].astype(str)
df['TargetSubType']=df['TargetSubType1'].astype(str)+" "+df['TargetSubType2'].astype(str)+" "+df['TargetSubType3'].astype(str)
df['TargetType']=df['TargetType1'].astype(str)+" "+df['TargetType2'].astype(str)+" "+df['TargetType3'].astype(str)
df['Nationality']=df['Natlty1'].astype(str)+" "+df['Natlty2'].astype(str)+" "+df['Natlty3'].astype(str)
df['TotalCasuality']=df['nkill']+df['nwound']+df['nkillus']+df['nwoundus']
df['ClaimMode']=df['ClaimMode1'].astype(str)+" "+df['ClaimMode2'].astype(str)+" "+df['ClaimMode3'].astype(str)

In [11]:
## Now some of the columns which need to be deleted
delete_col=['gsubname','gsubname2','gsubname3','gname','gname2','gname3','WeapType1','WeapType2','WeapType3','WeapType4','WeapSubType1','WeapSubType2','WeapSubType3',
            'WeapSubType4','AttackType1','AttackType2','AttackType3','target1','target2','target3','TargetSubType1','TargetSubType2','TargetSubType3','TargetType1',
            'TargetType2','TargetType3','Natlty1','Natlty2','Natlty3','nkill','nwound','nkillus','nwoundus','ClaimMode1','ClaimMode2','ClaimMode3','eventid']

df.drop(delete_col,inplace=True,axis=1)

In [12]:
df.head(1)

,Year,Month,Day,approxdate,extended,resolution,Country,Region,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,Alternative,multiple,success,suicide,corp1,corp2,corp3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claim2,claim3,compclaim,weapdetail,nkillter,nwoundte,property,PropeRXtent,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,GroupSubName,GroupName,WeponType,WeponSubType,AttackType,Target,TargetSubType,TargetType,Nationality,TotalCasuality,ClaimMode
0,1970,7,2,NaN,0,NaN,Dominican Republic,Central America & Caribbean,NaN,Santo Domingo,18.456792,-69.951164,1.0,0,NaN,NaN,1,1,1,0.0,NaN,0.0,1,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN,nan nan nan,MANO-D nan nan,Unknown nan nan nan,nan nan nan nan,Assassination nan nan,Julio Guzman nan nan,Named Civilian nan nan,Private Citizens & Property nan nan,Dominican Republic nan nan,NaN,nan nan nan


In [13]:
## Lets check the null values 

missing_data=df.isnull().sum().sort_values(ascending=False)
percentage_missing_data=(df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
info=pd.concat([missing_data,percentage_missing_data],keys=['Missing_value_count','Missing_percentage'],axis=1,ignore_index=False)

info

,Missing_value_count,Missing_percentage
divert,176824,0.998210
claim3,176824,0.998210
guncertain3,176822,0.998199
ransomnote,176630,0.997115
ransompaidus,176589,0.996884
ransomamtus,176578,0.996822
ransompaid,176367,0.995631
corp3,176122,0.994248
ransomamt,175793,0.992390
claim2,175315,0.989692


In [14]:
## Lets select the columns where NAN value is greater than or equal to 90 percent
col_greater=[]
col_lesseer=[]
for i in info.index:
  if(info['Missing_percentage'][i]>=0.90):
    col_greater.append(i)
  else:
    col_lesseer.append(i)



In [15]:
## Lets seperate the object and contionous column seperated
cat=[x for x in col_greater if df[x].dtype=='O']
con=[x for x in col_greater if x not in cat]

In [16]:
col_greater

['divert',
 'claim3',
 'guncertain3',
 'ransomnote',
 'ransompaidus',
 'ransomamtus',
 'ransompaid',
 'corp3',
 'ransomamt',
 'claim2',
 'guncertain2',
 'resolution',
 'kidhijcountry',
 'nhours',
 'compclaim',
 'ndays',
 'approxdate',
 'corp2',
 'nreleased',
 'hostkidoutcome_txt',
 'nhostkidus',
 'nhostkid']

In [17]:
## Since lattitude and longitude is related to City .Therefore we can fill the nan value with the help of city.
df['latitude'].isnull().sum() , df['longitude'].isnull().sum(),df['city'].isnull().sum()

(4369, 4370, 434)

In [18]:
## For latitude
df.loc[df['latitude'].isnull(),'LattiMiss']=1
df.loc[df['latitude'].notnull(),'LattiMiss']=0

## For longitude

df.loc[df['longitude'].isnull(),'LongiMiss']=1
df.loc[df['longitude'].notnull(),'LongiMiss']=0

## For City

df.loc[df['city'].isnull(),'city_miss']=1
df.loc[df['city'].notnull(),'city_miss']=0

In [19]:
df.loc[(df['LattiMiss']==1) & (df['LongiMiss']==1)& (df['city_miss']==1)].shape
# Lets drop the rows where all latitude longitude and city missing.
df_new=df.drop(df[(df['LattiMiss']==1) & (df['LongiMiss']==1)& (df['city_miss']==1)].index)

In [20]:
df_new['latitude'].isnull().sum() , df_new['longitude'].isnull().sum() , df_new['city'].isnull().sum()

(4361, 4362, 426)

In [21]:
df_new.loc[(df_new['LattiMiss']==0) & (df_new['LongiMiss']==1),:]

,Year,Month,Day,approxdate,extended,resolution,Country,Region,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,Alternative,multiple,success,suicide,corp1,corp2,corp3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claim2,claim3,compclaim,weapdetail,nkillter,nwoundte,property,PropeRXtent,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,GroupSubName,GroupName,WeponType,WeponSubType,AttackType,Target,TargetSubType,TargetType,Nationality,TotalCasuality,ClaimMode,LattiMiss,LongiMiss,city_miss
53821,1992,11,6,NaN,0,NaN,Ghana,Sub-Saharan Africa,Greater Accra,Tema,5.666667,NaN,1.0,0,NaN,NaN,1,1,1,-9.0,NaN,0.0,1,0,Govt,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,Explosive,NaN,NaN,1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,0,-9,NaN,nan nan nan,Unknown nan nan,Explosives nan nan nan,Unknown Explosive Type nan nan nan,Bombing/Explosion nan nan,Street nan nan,"Public Area (garden, parking lot, garage, beac...",Private Citizens & Property nan nan,Ghana nan nan,NaN,nan nan nan,0.0,1.0,0.0


In [22]:
## Now the one country where longitude is missing fill with the help of GeoCode.
geolocator = Nominatim(user_agent="geoapiExercises")
city='Tema'
  
location = geolocator.geocode(city)
df_new.loc[(df_new['LattiMiss']==0) & (df_new['LongiMiss']==1),'longitude']=location.longitude

In [23]:
df_new['LattiMiss']=df_new['LattiMiss'].astype(int)
df_new['LongiMiss']=df_new['LongiMiss'].astype(int)


In [24]:
group_data=df_new.loc[(df_new['LattiMiss']==1) & (df_new['LongiMiss']==1)]

In [25]:
nan_latti=df_new.loc[(df_new['LattiMiss']==1) & (df_new['LongiMiss']==1)]
City_fill=list(nan_latti['city'].unique())

In [26]:
NanImputation_LattiLongi=pd.DataFrame(City_fill,columns=['city'])
NanImputation_LattiLongi['Latitude']=0
NanImputation_LattiLongi['Longitude']=0


In [27]:
# declare an empty list to store 
# latitude and longitude of values  
# of city column 
longitude = [] 
latitude = [] 
   
# function to find the coordinate 
# of a given city  
def findGeocode(City): 
       
    # try and catch is used to overcome 
    # the exception thrown by geolocator 
    # using geocodertimedout   
    try: 
          
        # Specify the user_agent as your 
        # app name it should not be none 
        geolocator = Nominatim(user_agent="geoapiExercises") 
          
        return geolocator.geocode(City,timeout=None)

    except GeocoderTimedOut: 
          
        return findGeocode(City)
        

In [28]:
longitude = [] 
latitude = [] 
for i in NanImputation_LattiLongi.city:
  if findGeocode(i)!=None:
    loc=findGeocode(i)
    latitude.append(loc.latitude)
    longitude.append(loc.longitude)
  else:
    latitude.append(np.nan)
    longitude.append(np.nan)

In [29]:
NanImputation_LattiLongi['Latitude']=latitude
NanImputation_LattiLongi['Longitude']=longitude

In [30]:
NanImputation_LattiLongi.shape

(1866, 3)

In [31]:
NanImputation_LattiLongi.dropna(inplace=True)

In [32]:
# imputing the NAN value with the help of df dataframe

for index,rows in NanImputation_LattiLongi.iterrows():
  city=rows['city']
  lati=rows['Latitude']
  Longi=rows['Longitude']

  df_new.loc[(df_new['LattiMiss']==1) & (df_new['city']==city),'latitude']=lati
  df_new.loc[(df_new['LongiMiss']==1) & (df_new['city']==city),'longitude']=Longi

In [33]:
df_new['latitude'].isnull().sum() , df_new['longitude'].isnull().sum()

(890, 890)

In [34]:
df_new=df_new[df_new['longitude'].notna()]
df_new=df_new[df_new['latitude'].notna()]

In [35]:
df_new['latitude'].isnull().sum() , df_new['longitude'].isnull().sum() 

(0, 0)

In [36]:
df_new.to_csv('Terror.csv',index=False)